# 080. seq2seq language translation

### Encoder-Decoder model


- 영어-한국어 번역

한글 Glove file download

## 입력 data list 작성  

### 1. input_texts     : original language 의 input text  


### 2. Teacher Forcing 용 input / target data 생성

- target_texts_inputs  : 1 만큼 offset 된 target language sentence $\rightarrow$ `<sos>....`  
- target_texts  : target language sentence  $\rightarrow$ `.....<eos>`



- data 는 http://www.manythings.org/anki/  (Tab-delimited Bilingual Sentence Pairs) 에서 download  


    - English(input) + `\t` + The Other Language(target) + `\t` + Attribution(기여자) 형식으로 구성
        ex)
        - Hi.	안녕.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #8355888 (Eunhee).
        - I like horses.	나는 말을 좋아해.	CC-BY 2.0 (France) Attribution: tatoeba.org #1331062 (Kirschen112) & #8365125 (Eunhee)
        - We had fun with Tom.	우리는 톰과 즐거운 시간을 가졌다.	CC-BY 2.0 (France) Attribution: tatoeba.org #6845055 (CK) & #6845295 (dalgarak)

        
   

- teacher forcing 용 input, target 작성

In [ ]:
# load in the data
# load data
    # input 과 target translation 구분
    # target input 과 output 을 teacher forcing 입력 구성

## Tokenization

- language 가 2 개 이므로 언어별로 서로 다른 tokenizer 생성. 따라서, 2 개의 word_index 구성

### 영어 input text 의 tokenizer

In [ ]:
# 영어 텍스트 데이터를 위한 Tokenizer 객체 생성 (최대 단어 수 지정)
# 영어 텍스트 데이터를 사용하여 토크나이저 학습 수행 (단어 인덱스 구축)
# 영어 텍스트 데이터를 시퀀스로 변환
# 1500번째 시퀀스 출력
# 1500번째 원본 영어 텍스트 출력
# 1500번째 시퀀스의 각 인덱스를 단어로 변환하여 리스트로 출력

In [ ]:
# 영어 단어를 인덱스로 매핑한 사전 가져오기
# 고유한 입력 토큰 수 출력
# 사용할 단어 수 계산 (최대 어휘 크기와 실제 단어 수 + 1 중 작은 값)
# 입력 텍스트의 최대 길이 계산

### 한국어 Input Text

### Translation language 의 tokenizer
- 주의 사항 : $<sos>, <eos>$때문에 special character 를 filtering 하면 안됨

In [ ]:
# 한국어 텍스트 데이터를 위한 Tokenizer 객체 생성 (최대 단어 수 지정, 필터 비활성화)
# 한국어 입력 및 타겟 텍스트 데이터를 사용하여 토크나이저 학습 수행 (단어 인덱스 구축)
# 한국어 입력 텍스트 데이터를 시퀀스로 변환
# 한국어 타겟 텍스트 데이터를 시퀀스로 변환
# 1500번째 한국어 입력 시퀀스 출력
# 1500번째 한국어 타겟 시퀀스 출력
# 1500번째 한국어 입력 시퀀스의 각 인덱스를 단어로 변환하여 리스트로 출력
# 1500번째 한국어 타겟 시퀀스의 각 인덱스를 단어로 변환하여 리스트로 출력

In [ ]:
# 한국어 단어를 인덱스로 매핑한 사전 가져오기
# 고유한 출력 토큰 수 출력
# 사용할 단어 수 계산 (실제 단어 수 + 1)
# 타겟 텍스트의 최대 길이 계산

## sequence padding


### 주의 사항
- encoder 는 thought vector 생성 목적이므로 `pre`(default) 로 padding


- decoder 는 teacher forcing 을 해야하므로 `post` 로 padding

In [ ]:
# 영어 시퀀스를 패딩 처리하여 인코더 입력으로 변환 (최대 길이 max_len_eng로 설정)
# 한국어 입력 시퀀스를 패딩 처리하여 디코더 입력으로 변환 (최대 길이 max_len_kor로 설정, 'post' 방식으로 패딩)
# 한국어 타겟 시퀀스를 패딩 처리하여 디코더 타겟으로 변환 (최대 길이 max_len_kor로 설정, 'post' 방식으로 패딩)

## pretrained word embedding 값을 transfer learning

- Embedding layer 의 weight 를 pre-trained model 로 초기화  


- glove.6B 의 EMBEDDING_DIM version 사용  
    - space 로 구분된 text file
    - 첫번째는 word 이고 두번째 이후는 weight vector 값이다


- word index 가 1 부터 시작하므로 0 padding 감안하여 num_words 는 len(word2idx)+1, 혹은 MAX_VOCAB_SIZE 중 작은 것 선택  


- embedding_dict dictionary : key - word, value - embedding_vector

In [ ]:
def make_embedding(num_words, embedding_dim, tokenizer, max_vocab_size):
    # 임베딩 사전을 저장할 딕셔너리 초기화
    # GloVe 임베딩 파일 경로 설정
    # GloVe 임베딩 파일 열기
        # 파일의 각 줄에 대해 반복
            # 두번째 값부터 마지막 값까지는 해당 단어의 임베딩 벡터 값
    # 임베딩 매트릭스 초기화 (0으로 초기화)
    # 토크나이저의 단어 인덱스를 사용하여 임베딩 매트릭스 채우기

In [ ]:
# 사용할 단어 수 계산 (최대 어휘 크기와 실제 단어 수 + 1 중 작은 값)

### embedding layer 작성

- encoder 와 decoder 의 Embedding layer 에 pre-trained embedding weight 를 초기값으로 load

In [ ]:
# 임베딩 레이어 생성
# 영어 단어에 대한 임베딩 매트릭스 생성
# 임베딩 레이어 생성

## Build model

- encoder 와 decoder 의 embedding, lstm 및 dense layer 를 training 할 목적의 model 작성  

- encoder 는 decoder 에 states [h, c] 만 전달

- prediction 을 위한 model 은 training model 에서 만들어진 layer 들의 weight 를 이용하여 별도 작성  

## Encoder model 작성  

- Training과 Inference State 에서 모두 사용

In [ ]:
# 인코더 정의
# 인코더 입력 정의 (최대 길이를 max_len_eng로 설정)
# 사전 학습된 임베딩 레이어 사용
# LSTM 레이어를 사용하여 인코더 출력과 상태(hidden state와 cell state) 계산
# 인코더는 hidden state와 cell state만 디코더로 전달 (thought vector)
# 인코더 모델 정의
# 인코더 모델 요약 출력

## Decoder model for Teacher Forcing

- Training stage 에서 만 사용

In [ ]:
# 디코더 정의
# 디코더 입력 정의 (최대 길이를 max_len_kor로 설정)
# 디코더의 단어 임베딩은 사전 학습된 벡터를 사용하지 않음
# 교사 강요(Teacher-forcing)를 위한 디코더 LSTM 정의
# 초기 상태를 인코더의 [h, c] 상태로 설정
# 최종 Dense 레이어 정의
# Teacher-forcing 모델 생성
# 모델 컴파일 및 훈련 설정
# 모델 요약 출력

- teacher-forcing model 시각화

- teacher-forcing model train

- accuracy, loss 시각화

In [ ]:
# plot some data
# accuracies

- save model

## Make Predictions - Inference phase

- prediction 을 위한 별도의 decoder model 작성  

- 별로 training 없이 앞에서 train 된 weights 를 모두 재사용 한다.

- decoder 는 encoder 와 분리되어 구성

In [ ]:
# 추론(Inference)용 디코더 정의
# 디코더 LSTM의 hidden state 입력 정의 (512 차원)
# 디코더 LSTM의 cell state 입력 정의 (512 차원)
# 디코더 상태 입력 리스트 생성
# 디코더의 단일 단어 입력 정의 (1차원)
# 단일 단어 입력을 임베딩 벡터로 변환
# 출력과 hidden states 저장
# 새로운 디코더 상태 리스트 생성
# 디코더 출력에 Dense 레이어 적용
# 추론용 디코더 모델 생성
# 추론용 디코더 모델 요약 출력

- inference 용 decoder model 시각화

예측하는 동안 index를 단어를 되돌리기 위한 reverse word2idx dictionary 작성

### Translation Inference

In [ ]:
def decode_sequence(input_seq):
    # 인코더 모델을 이용하여 입력을 상태 벡터로 인코딩
    # 길이 1의 빈 타겟 시퀀스 생성
    # 타겟 시퀀스의 첫 번째 단어를 시작 토큰(<sos>)으로 설정
    # 종료 토큰(<eos>)이 디코딩에서 생성되면 루프 종료
    # 번역문 생성
        # 디코더 모델을 이용하여 다음 단어 예측 및 상태 업데이트
        # argmax로 가장 확률 높은 단어 선택 (탐욕적 선택)
        # 생성된 단어를 디코더의 다음 입력으로 사용
        # 상태 업데이트
# 번역 테스트

### 임의의 영어 입력을 처리하는 영한 번역 함수

In [ ]:
def Eng_Kor_translation(txt):
    # 입력된 영어 텍스트를 시퀀스로 변환
    # 시퀀스를 패딩 처리하여 인코더 입력으로 변환 (최대 길이를 max_len_eng로 설정)
    # 인코더 입력을 디코더 시퀀스로 변환하여 번역 결과 반환